## Название документа

Название НПА – это краткий текст, описывающий основную суть и тему документа.
Название часто начинается с предлогов «о» или «об». В тексте НПА название может встречаться
в кавычках или без кавычек. Название не включает в себя тип НПА.

Для примера название документа – это «О заместителе Министра внутренних дел
Российской Федерации – начальнике Следственного департамента Министерства внутренних дел Российской Федерации».

Поле name должно содержать строку с названием в кавычках или без, в зависимости от того, как оно встретилось в тексте НПА. При сравнении предсказанного и настоящего номера в проверяющей системе не учитывается регистр букв, пробелы и символы перевода строки не различаются.

## Орган, принявший акт

Часто автор документа указывается после слова «Принят», но далеко не всегда, например в указе автор - «Президент Российской Федерации».

В рамках задания необходимо для каждого документа определить орган, его принявший. Наименование органов необходимо извлекать в «нормализованном виде»:

* Ярославская областная дума
* Федеральная служба по надзору в сфере образования и науки
* Правительство Удмуртской Республики
* Департамент охраны окружающей среды и природопользования Ярославской области

В обучающей и тестовой выборке около 200 уникальных органов. В тестовой выборке есть органы, которых нет в обучающей.

Поле authority должно содержать наименование органа в нормализованном виде. При сравнении предсказанного и настоящего органа не учитывается регистр букв, пробелы и символы перевода строки не различаются.

### Оценка качества

Допускается нечеткое совпадение. Метрика качества – средний коэффициент Жаккара.

Коэффициент Жаккара вычисляется следующим образом:

$$ L(s_{extr}, s_{gold}) = \frac{intersect}{offset_{start} + intersect + offset_{end}} $$

где $intersect$ - длина самой длинной общей подстроки у выделенной строки и строки из «золотой метки», $offset_{start}$ - наибольшее количество символов до начала общей подстроки в выделенной строке и в строке из «золотой метки», $offset_{end}$ - наибольшее количество символов от конца общей подстроки до конца строки в выделенной строке и в строке из «золотой метки».

Эта метрика принимает значения от 0 до 1 и штрафует как за выделение более короткой строки чем строка из «золотой метки», так и за выделение более длинной строки.

In [1]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsNERTagger,
    NamesExtractor,
    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

Doc(text='е 2. Признать утратившим силу приказ Министерства...)

In [35]:
with open('train/txts/dfc5cb33a22b9c7864bffa24b8a7e26aff4024c8.txt', 'r') as f:
    text = f.read()
doc = Doc(text) 
doc.segment(segmenter)
doc.tag_ner(ner_tagger)
display(doc.spans[:5])
for span in doc.spans:
    span.normalize(morph_vocab)
    
{_.text: _.normal for _ in doc.spans}
text

[DocSpan(start=14, stop=34, type='LOC', text='РОССИЙСКОЙ ФЕДЕРАЦИИ', tokens=[...]),
 DocSpan(start=78, stop=84, type='LOC', text='МОСКВА', tokens=[...]),
 DocSpan(start=136, stop=173, type='ORG', text='Федеральном агентстве водных ресурсов', tokens=[...]),
 DocSpan(start=175, stop=188, type='LOC', text='Правительство', tokens=[...]),
 DocSpan(start=189, stop=209, type='LOC', text='Российской Федерации', tokens=[...])]

'ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ\nПОСТАНОВЛЕНИЕ\n\nот 21 апреля 2014 г. № 362\n\nМОСКВА\n\nО внесении изменения в подпункт 5.1.3 Положения\nо Федеральном агентстве водных ресурсов\n\nПравительство Российской Федерации п остановляет:\n\n1. Нодпункт 5.1.3 Положения о Федеральном агентстве водных\nресурсов, утвержденного постановлением Правительства Российской\nФедерации от 16 июня 2004 г. № 282 "Об утверждении Положения\nо Федеральном агентстве водных ресурсов" (Собрание законодательства\nРоссийской Федерации, 2004, № 25, ст.2564; 2006, № 52, ст. 5599),\nдополнить словами ", а также в отношении водных объектов, по которым\nпроходит государственная граница Российской Федерации".\n\n2. Реализация настоящего постановления осуществляется\nФедеральным агентством водных ресурсов в пределах установленной\nПравительством - Российской Федерации - предельной - численности\nработников его центрального аппарата и территориальных органов,\nа также - бюджетных - ассигнований, - предусмотренных - Аг

In [13]:
import re
from pymystem3 import Mystem

organizations = ['правительство', 'губернатор', 'администрация', 'президента']
def extract_authority(doc):
    doc = doc.lower()
    authority = ""
    for organization in organizations:
        index = doc.find(organization)
        if index != -1:
            doc = doc[index:]
            authority = doc.split('\n')[0]
            if authority.startswith('президента'):
                authority = re.sub('президента', 'президент', authority)
            return authority
    index = doc.find('принят')
    if index != -1:
        doc = doc[index + 6:]
        authority = doc.split('\n')[0]
        authority = re.sub(r'\d.*', '', authority)
#         authority = re.sub(r'[^а-я ]', '', authority)
#         m = Mystem()
#         authority = ''.join(m.lemmatize(authority))
    return authority

def extract_name(doc):
    doc = doc.lower()
    name = re.search(r'\n((о|об) ([^\n]+\n)+)\n', doc)
    if name:
        return name.group(1)
    else:
        return ""

In [14]:
def predict(test):
    results = []
    for doc in test:
        authority = extract_authority(doc)
        name = extract_name(doc)
        prediction = {"type": "",
                      "date": "",
                      "number": "",
                      "authority": authority,
                      "name": name}
        results.append(prediction)
    return results

In [15]:
from eval_module import quality
import json

train = []
labeled = []
names = []
with open("train/gold_labels.txt", "r") as read_file:
    for doc_info in read_file.readlines():
        doc_dict = json.loads(doc_info)
        docname = 'train/txts/' + doc_dict['id'] + '.txt'
        names.append(docname)
        with open(docname, 'r') as f:
            train.append(f.read())
        labeled.append(doc_dict['label'])

predicted = predict(train)
quality(predicted, labeled)

{'date_accuracy': 0.0,
 'number_accuracy': 0.0,
 'type_f1_score': 0.0,
 'name_jaccard': 0.9330221188,
 'authority_jaccard': 0.7825957161,
 'subtasks_improves': 2}

In [65]:
import numpy as np

predicted_labels = np.array(predicted)
test_labels = np.array(labeled)
names = np.array(names)

doc_type = 'постановление'
true_types = np.array([x['type'] for x in labeled])
ids = np.where(true_types == doc_type)

y_test = [(x['authority'], x['type']) for x in test_labels[ids]]
y_pred = [x['authority'] for x in predicted_labels[ids]]

wrong_docs = []
i = 0
for test, pred in zip(y_test, y_pred):
    if test[0].lower() != pred.lower():
        wrong_docs.append((names[ids][i], test, pred))
    i += 1
    
for doc_name, test, pred in wrong_docs:
    with open(doc_name, 'r') as f:
        print(f"right value = {test}")
        print(f"wrong value = {pred}")
        print(f"docname = {doc_name}")
        print(f.read())
        print("=====")

right value = ('Конституционный Суд Российской Федерации', 'постановление')
wrong value = 
docname = train/txts/5cc4127c5b875ef6b812c5a9a4285ba4a936947c.txt
Именем

Российской Федерации

ПОСТАНОВЛЕНИЕ
КОНСТИТУЦИОННОГО СУДА РОССИЙСКОЙ ФЕДЕРАЦИИ

по делу о проверке конституционности статей 4, 5 и пункта 5 статьи
14 Федерального закона "О предупреждении распространения
туберкулеза в Российской Федерации" в связи с запросами
Правительства - Волгоградской - области и - Правительства
Сахалинской области

город Санкт-Петербург 24 декабря 2013 года

Конституционный Суд Российской Федерации в составе Председателя
В.Д.Зорькина, - судей - К.В.Арановского,  А.И.Бойцова, - Н.С.Бондаря,
Ю.М.Данилова, Л.М.Жарковой, Г.А.Жилина, С.М.Казанцева,
М.И.Клеандрова, С.Д.Князева, А.Н.Кокотова, Л.О.Красавчиковой,
С.П.Маврина, Н.В.Мельникова, Ю.Д.Рудкина, Н.В..Селезнева,
О.С.Хохряковой, В.Г.Ярославцева,

с участием представителя Правительства Волгоградской области
В.В.Черячукина, полномочного представителя Госуд

ПОСТАНОВЛЕНИЕ
ГУБЕРНАТОРА СТАВРОПОЛЬСКОГО КРАЯ

09 июля 2015 г. г. Ставрополь № 352

Об установлении ограничительных мероприятий (карантина)
на территории села Курсавка,
Андроповский район

В соответствии со статьей 17 Закона Российской Федерации "О ветеринарии", в связи с возникновением очага бешенства, выявленного у дикого
плотоядного животного (лисы) (далее - очаг бешенства) на подворье в селе
Курсавка (ул.8 Марта, 22), Андроповский район, на основании представления
начальника управления ветеринарии Ставропольского края Трегубова А.Н.
от 01.07.2015 № 01-04/2839 об установлении ограничительных мероприятий
(карантина) на территории села Курсавка, Андроповский район, в целях ликвидации очага бешенства и недопущения распространения заболевания на
территории Ставропольского края

ПОСТАНОВЛЯЮ:

1. Установить ограничительные мероприятия (карантин) на территории
села Курсавка, Андроповский район, Ставропольский край, в пределах
ул.8 Марта, дома 14, 15, 16, 18, 19, 22, ул.Братской, дома 28, 